<a href="https://colab.research.google.com/github/shubham-k01/D2K-Text-Summarisation/blob/main/D2K_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
with open('/content/PS3_Dataset.txt') as f:
  l = f.read()
  # l = l.replace(' ',',')
  print(l)

https://pubs.acs.org/doi/10.1021/acs.chemrev.1c00107

https://downloads.hindawi.com/journals/cin/2022/5624475.pdf

https://pubs.acs.org/doi/pdf/10.1021/acssynbio.2c00015

https://downloads.hindawi.com/journals/cin/2022/5624475.pdf

https://link.springer.com/article/10.1007/s11023-021-09573-8

https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0237749&type=printable

https://www.mdpi.com/2079-4991/12/15/2646

https://pubs.acs.org/doi/10.1021/acs.chemrev.0c00004

https://www.hindawi.com/journals/jhe/2022/4653923/

https://www.mdpi.com/1660-4601/19/22/15027

https://www.nature.com/articles/s41598-021-84637-4

https://onlinelibrary.wiley.com/doi/10.1111/1754-9485.13274

https://www.mdpi.com/1424-8220/22/23/9148

https://www.frontiersin.org/articles/10.3389/fnagi.2021.633752/full

https://pubs.asahq.org/anesthesiology/article/132/2/379/108833/Artificial-Intelligence-in-AnesthesiologyCurrent

https://www.mdpi.com/2072-6694/14/6/1524

https://translational-medicine.biomedc

In [ ]:
!pip install metapub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
NCBI_API_KEY= '542e5c51b595a54f0089ff5b8ad5ddf1cb09 '

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.7 MB/s eta 0:00:00


In [ ]:
from Bio import Medline
import pandas as pd
from tqdm import tqdm

In [ ]:
data = []
with open('/content/pubmed-healthcare-set.txt',encoding='utf-8') as f:
  pmids = Medline.parse(f)
  for pmid in pmids:
    try:
      pid = pmid['PMID']
    except:
      pid = ''
    try:
      title = pmid['TI']
    except:
      title = ''
    try:
      abstract = pmid['AB']
    except:
      abstract = ''
    
    sample = {
        'PMID':pid,
        'Title':title,
        'Abstract':abstract
    }
    data.append(sample)

print(data)

[{'PMID': '32336480', 'Title': 'The Importance of Diversity and Inclusion in the Healthcare Workforce.', 'Abstract': 'BACKGROUND: Diversity and inclusion are terms that have been used widely in a variety of contexts, but these concepts have only been intertwined into the discussion in healthcare in the recent past. It is important to have a healthcare workforce which represents the tapestry of our communities as it relates to race/ethnicity, gender, sexual orientation, immigration status, physical disability status, and socioeconomic level to render the best possible care to our diverse patient populations. METHODS: We explore efforts by the Liaison Committee on Medical Education (LCME), the Institute of Medicine (IOM), and other medical organizations to improve diversity and inclusion in medicine. CONCLUSION: Finally, we report on best practices, frameworks, and strategies which have been utilized to improve diversity and inclusion in healthcare.'}, {'PMID': '31550922', 'Title': 'Arti

In [ ]:
!pip install sqlitedict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sqlitedict: filename=sqlitedict-2.1.0-py3-none-any.whl size=16869 sha256=3015c829e48bb5a10ddae9005076dbe05bec1660a54ed17215dc53963730ce7c
  Stored in directory: /root/.cache/pip/wheels/04/c6/16/46e174009277f9bccdaa7215a243939d2f70180804b249bf3a
Successfully built sqlitedict


In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

,PMID,Title,Abstract
0,32336480,The Importance of Diversity and Inclusion in t...,BACKGROUND: Diversity and inclusion are terms ...
1,31550922,Artificial intelligence in healthcare: An esse...,Artificial Intelligence (AI) is evolving rapid...
2,35670616,Healthcare Compensation Plans: Current Challen...,"The US healthcare sector differs from others, ..."
3,31732680,What does 'quality' add? Towards an ethics of ...,"In this paper, we argue that there are importa..."
4,36495531,Healthcare Reform Is Not Yet on the Horizon.,Healthcare reform is on everyone's lips these ...
5,33846028,Integrating planetary health into healthcare: ...,BACKGROUND: Anthropogenic climate change poses...
6,33386675,A systematic review of healthcare professional...,While technical and profession-specific compet...
7,34232148,What can a learning healthcare system teach us...,PURPOSE OF REVIEW: This review describes the l...
8,33995804,Challenges in healthcare financing for surgery...,One-third of the global burden of disease is a...
9,36194519,Achieving sustainable healthcare through depre...,


In [ ]:
import csv
import logging
from multiprocessing.pool import ThreadPool
from sys import stdin, stdout
from time import perf_counter
from typing import Iterator, List, Optional

In [ ]:
from metapub import PubMedArticle, PubMedFetcher
from sqlitedict import SqliteDict

# shared by members of the ThreadPool 
article_cache = SqliteDict("pubmed_cache.db", "articles")
fetcher = PubMedFetcher()


def fetch_article(pmid: str) -> PubMedArticle:
    "call fetcher.article_by_pmid with lots of logging statements"
    # log as this is going to take a non-trivial amount of time
    logging.info("fetching article with pmid=%r", pmid)

    t0 = perf_counter()
    article = fetcher.article_by_pmid(pmid)
    dt = perf_counter() - t0
    logging.info("fetched pmid=%s in %.3fms", pmid, dt * 1000)

    # check if we got the right thing back
    if article.pmid != pmid:
        logging.warning("article with pmid=%r returned pmid=%r", pmid, article.pmid)

    return article


def fetch_article_cached(pmid: str) -> PubMedArticle:
    """cache XML of fetched articles so subsequent runs are faster.
    errors are returned as None
    """
    try:
        xml = article_cache[pmid]
    except KeyError:
        # cached failed, continue below with fetching
        pass
    else:
        # cache hit, parse it
        return PubMedArticle(xml)

    try:
        article = fetch_article(pmid)
    except Exception as err:
        logging.warning("error fetching pmid=%r: %s", pmid, err)
        article = None

    # save XML in cache for next time
    article_cache[pmid] = article.xml
    article_cache.commit()

    return article


def fetch_articles(
    pmids: List[str], *, processes: Optional[int] = None
) -> Iterator[PubMedArticle]:
    "use a threadpool to fetch articles from metapub in parallel, caching where possible"
    with ThreadPool(processes=processes) as pool:
        for article in pool.imap_unordered(fetch_article_cached, pmids):
            # couldn't be fetched
            if article is None:
                continue
            yield article


def main() -> None:
    "fetch articles by PMID from stdin and write to stdout in CSV format"
    pmids = stdin.read().split()

    out = csv.writer(stdout)
    first = True
    for article in fetch_articles(pmids, processes=6):
        row = dict(
            pmid=article.pmid,
            pmc=article.pmc,
            doi=article.doi,
            date=article.history["pubmed"],
            title=article.title,
            authors=article.authors,
            journal=article.journal,
            abstract=article.abstract
        )
        if first:
            out.writerow(row.keys())
            first = False
        out.writerow(row.values())


if __name__ == "__main__":
    logging.basicConfig(level=logging.DEBUG)
    main()

In [ ]:
main()

In [ ]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 8.6 MB/s eta 0:00:00


In [ ]:

# importing required modules
from PyPDF2 import PdfReader
 
# creating a pdf reader object
reader = PdfReader('sample.pdf')
 
# printing number of pages in pdf file
print(len(reader.pages))
 
# getting a specific page from the pdf file
page = reader.pages[0]
 
# extracting text from page
text = page.extract_text()
print(text)

21
Health Policy and Technology  10 (2021) 209–229  
Contents  lists available  at ScienceDirect  
Health  Policy  and Technology  
journal  homepage:  www.elsevier.com/locate/hlpt  
Literature  Review  
Telemedicine  and health  policy:  A systematic  review  
Clemens  Scott Kruse  ∗, Kelly Williams  1 , John Bohls  1 , Waleed  Shamsi  1 
School of Health Administration,  Texas State University,  San Marcos, TX, United States 
a r t i c l e i n f o 
Article history: 
Available  online 29 October 2020 
Keywords:  
Health 
Policy 
Telemedicine  
Access 
Health policy a b s t r a c t 
Background:  Telemedicine  diagnoses  and treats patients  remotely  via telecommunications  technology  all 
over the world. Telemedicine  becomes  more prevalent  as providers  recognize  the beneﬁts,  patients  re- 
ceive increased  access and payers see the reduction  in cost of care. 
Objective:  Telemedicine  studies have shown success in limiting  geographical  constraints,  time spent, and 
costs in